In [3]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from scipy.optimize import minimize
from sklearn import clone
from tqdm import tqdm
import funcy

In [4]:
%load_ext autoreload
%autoreload 2

In [6]:
from code.helpers import plotDecisionBoundary

/Users/aciupan/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/aciupan/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [7]:
from code.helpers import read_in, plotDecisionBoundary, make_fname
from code.logistic_regression import nll, l1_reg, l2_reg, LogReg
from code.gradient_descent import gradient_descent

SyntaxError: Missing parentheses in call to 'print' (logistic_regression.py, line 20)

# 1. Logistic Regression
1. Note: Sklearn uses C = 1/λ
2. See `intercept_scaling` in sklearn.
3. When you report weights, include `W0`

In [18]:
%%latex
$ NLL(w, w0) =  \sum(log (1 + exp  (−y(i)(wx(i) + w0))))$

<IPython.core.display.Latex object>

### TODO: 
2. Why different coeffs than sklearn?
3. Compare L=1 to L=0
4. Results tables
5. Decision boundaries?


In [57]:
X,y = read_in(data=3)

In [64]:
#clf = LogReg(reg_func=l2_reg, L=1.).fit(X, y)

In [59]:
nll(X, y, clf.coef_)

0.0


160.48492780042571

In [60]:
clf.predict_proba(X).round(2)

array([ 0.97,  0.95,  0.15,  0.06,  0.15,  0.94,  0.17,  0.78,  0.07,
        0.03,  0.16,  0.27,  0.11,  0.33,  0.91,  0.94,  0.06,  0.23,
        0.88,  0.72,  0.97,  0.99,  0.17,  0.95,  0.07,  0.1 ,  0.03,
        0.98,  0.07,  0.61,  0.09,  0.6 ,  0.05,  0.31,  0.08,  0.85,
        0.13,  0.05,  0.93,  0.1 ,  0.21,  0.13,  0.99,  0.05,  0.09,
        0.79,  0.82,  0.06,  0.09,  0.05,  0.1 ,  0.02,  0.07,  0.02,
        0.19,  0.96,  0.69,  0.11,  0.9 ,  0.36,  0.96,  0.93,  0.62,
        0.21,  0.07,  0.1 ,  0.07,  0.91,  0.87,  0.31,  0.05,  0.75,
        0.95,  0.04,  0.95,  0.05,  0.53,  0.11,  0.98,  0.06,  0.35,
        0.08,  0.68,  0.92,  0.03,  0.97,  0.98,  0.06,  0.98,  0.09,
        0.17,  0.12,  0.83,  0.03,  0.08,  0.18,  0.66,  0.87,  0.08,
        0.27,  0.05,  0.13,  0.08,  0.08,  0.83,  0.15,  0.98,  0.81,
        0.88,  0.15,  0.8 ,  0.09,  0.98,  0.07,  0.98,  0.95,  0.2 ,
        0.04,  0.93,  0.97,  0.97,  0.12,  0.07,  0.13,  0.09,  0.03,
        0.86,  0.16,

In [61]:
nll(X,y, np.squeeze(log.coef_))

0.0


168.02852082848531

In [53]:
log.coef_

array([[-0.03374817,  7.15227365]])

In [ ]:
log.predict_proba()

In [39]:
X.shape

(200, 2)

In [24]:
#clf.optim_

In [25]:
probas = clf.predict_proba(X)

In [29]:
clf.score(X, y)

1.0

In [47]:
log = LogisticRegression( C=1e6).fit(X, y)

In [36]:
LogisticRegression?

In [48]:
log.coef_

array([[-0.03374817,  7.15227365]])

In [49]:
log.score(X, y)

0.97499999999999998

In [ ]:
log.pr

In [15]:
clf.optim_

132.88539681960287

yhat = clf.predict(X)
(np.sign(yhat) == y).mean()

0.94499999999999995

In [ ]:
clf = LogisticRegression(fit_intercept=True, C=1e9).fit(X, Y)
yhat = clf.predict(X)
cof = np.squeeze(clf.coef_)
cof

In [52]:
l1_log = LogisticRegression(C=.01, penalty='l1')
l2_log = LogisticRegression(C=.01, penalty='l2')
lr_no_reg = LogisticRegression(C=1e12, penalty='l2')
#.fit(X,Y).coef_

In [53]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [54]:
def all_stats():
    res ={}
    for d in [1,2,3,4]:
        for C in [.0001, .001, .01,1, 1e12]:
            for pen in [ 'l2']:
                clf = LogisticRegression(C=C, penalty=pen)
                #clf = LogisticRegressionCV(penalty=pen)

                res[(d,C, pen)] = get_stats(d, clf)
    return res


def get_stats(data, clf):
    X, Y = read_in(data=data, suffix='train')
    
    Xt, Yt = read_in(data=data, suffix='test')
    Xv, Yv = read_in(data=data, suffix='validate')
    clf.fit(X, Y)
    return pd.Series(dict(te_score = clf.score(Xt, Yt),
                          tr_score = clf.score(X, Y),
                          val_score = clf.score(Xv, Yv),
                          weight_norm  = np.sum(np.abs(clf.coef_)),
                          clf=clf))

In [55]:
res  =all_stats()

In [56]:
clf = pd.DataFrame(res).rename_axis(['data', 'C', 'Penalty'], 1).loc['clf'].iloc[0]#.get_params()#T[['tr_score', 'oos_score', 'weight_norm']].round(2)

In [58]:
tab1 = pd.DataFrame(res).rename_axis(['data', 'C', 'Penalty'], 1)
    
tab1.loc['clf'].iloc[0]#.get_params()#T[['tr_score', 'oos_score', 'weight_norm']].round(2)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [61]:
tab1.drop('clf')

data                   1                                                      \
C           1.000000e-04 1.000000e-03 1.000000e-02 1.000000e+00 1.000000e+12   
Penalty               l2           l2           l2           l2           l2   
te_score           0.995        0.995        0.995            1            1   
tr_score            0.99         0.99       0.9925            1            1   
val_score              1            1            1            1            1   
weight_norm    0.0373217     0.279099      0.97279      3.55085       12.951   

data                   2                                                      \
C           1.000000e-04 1.000000e-03 1.000000e-02 1.000000e+00 1.000000e+12   
Penalty               l2           l2           l2           l2           l2   
te_score           0.805        0.805         0.81        0.805        0.805   
tr_score            0.82       0.8175       0.8225       0.8325         0.83   
val_score           0.82         0.82        0.815        0.825        0.825   
weight_norm    0.0184705      0.15842     0.719829      1.76958       1.8272   

data                   3                                                      \
C           1.000000e-04 1.000000e-03 1.000000e-02 1.000000e+00 1.000000e+12   
Penalty               l2           l2           l2           l2           l2   
te_score            0.96         0.96         0.96         0.97         0.95   
tr_score            0.96         0.96         0.96       0.9775       0.9875   
val_score           0.94         0.94         0.94         0.97        0.965   
weight_norm    0.0210195     0.188839      1.00797      5.07972      14.0761   

data                   4                                                      
C           1.000000e-04 1.000000e-03 1.000000e-02 1.000000e+00 1.000000e+12  
Penalty               l2           l2           l2           l2           l2  
te_score          0.5025        0.505          0.5       0.4975       0.4975  
tr_score            0.52       0.5175        0.515        0.515        0.515  
val_score         0.5075        0.505       0.4975       0.4925       0.4925  
weight_norm   0.00224741    0.0158508    0.0401831    0.0483625    0.0484622

In [ ]:
test = np.loadtxt('hw2_resources/data/data'+name+'_test.csv')
Xt = test[:,0:2]
onz = np.ones(Xt.shape[0])
Xt = np.hstack([Xt, onz.reshape(Xt.shape[0],1)])
Yt= np.ravel(test[:,2:3])


In [5]:
kk

NameError: name 'Xv' is not defined

In [ ]:

val_error(w_no)

In [ ]:
test_error(w_no)

In [ ]:
wL, pL = _gradient_descent(funcy.partial(nll, L=1.), init_weights=np.copy(w), lr=.01)

In [ ]:
val_error(wL)

In [ ]:
val_error(wL1)

In [ ]:
wL1, pL1 = _gradient_descent(funcy.partial(nll, L=1., reg_func=l1_reg),
                             init_weights=np.copy(w), lr=.01)

In [ ]:
val_error(wL1)

In [ ]:
ax = paths[['w_0', 'w_1', 'w_2']].plot()
ax.set_title('No Regularization')

In [ ]:
np.copy(w) - 1

In [ ]:
ax = pL[['w_0', 'w_1', 'w_2']].plot()
ax.set_title('L2 Loss')

In [ ]:
ax = pL1[['w_0', 'w_1', 'w_2']].plot()
ax.set_title('L1 Loss')

In [ ]:
pL1[['w_0', 'w_1', 'w_2']].diff().abs().tail(20).plot()

In [ ]:
%%latex
$\bf{1.1}$ With $\lambda$ = 0, the weight vector takes longer to converge, as the weights keep expanding away from 0 for longer.
When we add the $L2$ regularization penalty, with $\lambda$=1, it takes only 90 iterations to achieve convergence.

In [ ]:
%%latex
$\bf{1.2}$ When we add the $L1$ regularization penalty, with $\lambda$=1, it takes  more iterations to achieve convergence because the coefficients that drop out bounce around zero.


In [ ]:
%%time


# 2.1 Implement Dual Form of linear SVM with slack variables

### Intro Notes
- Weight vector w as weighted linear combination of instances
- Only points on margin matter; ignore the rest, solution remains
unchanged
- Keeps instances away from the margin
- "Finding the minimum error separating hyperplane is NP-hard" 

- f(x) = <w, x> + b


### TODO:
- verify get_theta division
- tables, figures for writeup



In [ ]:
%%latex
For linear SVM, $w = \sum{y_{i}a_{i}x_{i}}$.

In [ ]:
from cvxopt import matrix, solvers

In [ ]:
from numpy.linalg import norm

In [ ]:
#def train_svm(X, y, kernel=linear_kernel, C=1.):
n_samples, n_features = X.shape
y = Y

In [ ]:
one_vec = X.dot(X[0])
kernel  = linear_kernel
C = 1.
LINEAR_KERNEL = np.dotm

In [2]:
#P

In [ ]:
del get_weights

In [ ]:
def get_theta(sv, svy, ind, alpha, K, non_zero_mask):
        theta_0 = 0   #intercept
        for n in range(len(alpha)):
            theta_0 +=  (svy[n] - np.sum(alpha *svy * K[ind[n],  non_zero_mask]))
            theta_0 /= len(alpha)  # seems wierd
        return theta_0

In [63]:
class SVMD(object):
    def __init__(self, X, y, kernel=LINEAR_KERNEL, C=1.):
        n_samples, n_features = X.shape
        self.X =  X
        self.y = y
        self.C = C
        self.K = np.array([[kernel(X[i], X[j]) for i in range(n_samples)] for j in range(n_samples)])
        self.kernel = kernel
        self.n_features = X.shape[1]
        self.solution = self.train()
        alpha, ind, mask = self.inspect(self.solution)
        svx = self.X[mask]
        svy = self.y[mask]
        self.svx, self.svy, self.ind, self.alpha, self.mask = svx, svy, ind, alpha[mask], mask

    def fit(self):
        self.theta = get_theta(self.svx, self.svy,  self.ind, self.alpha, self.K, self.mask)
        return self
    
    def train(self):
        X,y, kernel, C = self.X, self.y, self.kernel, self.C  # to avoid typing self
        n_samples, n_features = X.shape
        P = matrix(np.outer(y,y) * self.K)
        q = matrix(np.ones(n_samples)*-1.)
        A = matrix(y, (1, n_samples))
        b = matrix(0.)
        G = matrix(np.vstack([np.diag(np.ones(n_samples)) * -1, np.diag(np.ones(n_samples))]))
        h = matrix(np.hstack([np.zeros(n_samples), self.C *np.ones(n_samples)]))
        return solvers.qp(P, q, G, h, A, b)

    
    @staticmethod
    def inspect(solution, cutoff=1e-5):
        alpha = np.ravel(solution['x'])
        non_zero_mask = alpha > cutoff
        ind = np.arange(len(alpha))[non_zero_mask]
        print ind.mean()
        return alpha,ind, non_zero_mask
    
    def predict(self, X_new):
        self.weights = (self.alpha*self.svy).dot(self.svx)
        self.margin = 1./ norm(self.weights)
        if self.kernel == linear_kernel:    
            return np.sign(np.dot(X_new, self.weights) + self.theta)
        else:
            pred_val = [np.sum([a*y*clf.kernel(X[i], x) for a,y,x in zip(clf.alpha, clf.svy, clf.svx)]) for i in range(X_new.shape[0])]
            self.margin = None
            return np.sign(np.array(pred_val) + self.theta)

    def score(self, X_new, y_new):
        yhat = self.predict(X_new)
        assert yhat.shape ==  y_new.shape, "Shape mismatch"
        return (yhat == y_new).mean()
    def plot_boundary(self, X, y, **kwargs):
        return plotDecisionBoundary(X,y, self.predict, [-1,0,1], **kwargs)

NameError: name 'LINEAR_KERNEL' is not defined

In [62]:
clf = SVMD(X, Y, kernel=gaussian_kernel, C=10.).fit()

NameError: name 'SVMD' is not defined

In [ ]:
Xt, Yt = read_in(data=4, suffix='test')


In [ ]:
def get_stats(data, C=1., **kwargs):
    X, Y = read_in(data=data, suffix='train')
    Xt, Yt = read_in(data=data, suffix='test')
    Xv, Yv = read_in(data=data, suffix='validate')
    clf = SVMD(X, Y, C=C, **kwargs).fit()
    assert clf.C == C, 'clf.c={}'.format(clf.C)
    clf.score(Xt, Yt)
    return pd.Series(dict(n_sv=len(clf.alpha),
                          tr_score = clf.score(X, Y),
                          te_score = clf.score(Xt, Yt),
                          val_score = clf.score(Xv, Yv),
                          margin=clf.margin,
                          weights = clf.weights,
                          clf=clf
                         )) 

del X, Y
X, Y = read_in(data=1, suffix='train')


In [ ]:
%%capture
res  = pd.DataFrame({d:get_stats(d) for d in [1,2,3,4]}).T

In [ ]:
# res.loc[1].clf.ind 
assert res.loc[1].clf.ind[0] ==16, 'something changed upstream'

In [ ]:
tab = 1- res.drop('clf', 1)[['tr_score', 'val_score']].rename_axis('2D Dataset')
tab.columns = ['Training Error', 'Validation Error']
#print tab.to_latex()
#tab

In [ ]:
def show_plots(res=res):
    suffix = 'train'
    for k, row in res.iterrows():
        v = row.clf    
        X, y = read_in(k, suffix=suffix)
        score = v.score(X,y)
        v.plot_boundary(X,y, 
                        title='{} data {}, score={}'.format(suffix, k, score, row.weights)
                       )
        pl.show()

In [ ]:
show_plots()

In [ ]:
res

### 2.3 Kernels


### 2.3.0 Notes
- Increasing C allows for more nonlinearities
- Decreases # errpors
- SV boundary may not be contiguous
- Kernel width adjusts function class

### 2.3.1 Extend code to handle Gaussian RBF kernel

**(A)** What happens to the geometric meargin `1/|w|` as `C`  increases? Will this always happen when we increase C?

**(B)** What happens to the number of support vectors as C increases?

**(C)**Why would maximizing the geometric margin `1/||w||` on the training set not be an appropriate criterion for selecting C? Is there an alternative criterion that we could use for this purpose?


In [ ]:
def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x,y, sigma=1.):
    return np.exp(-norm(x-y)**2 / 2 * (sigma **2))

In [ ]:
from ggplot import *


In [ ]:
X,Y = read_in(data=4, suffix='train')

In [ ]:
%%capture
clf = SVMD(X, Y, kernel=gaussian_kernel).fit()

In [ ]:
clf.score(clf.X, clf.y)

In [ ]:
 clf.kernel 

In [ ]:
%%capture
lin = SVMD(X, Y, kernel=LINEAR_KERNEL).fit()

In [ ]:
res

In [ ]:
X,y = read_in(3)

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
svm.coef_

In [ ]:
svm = LinearSVC(loss='hinge').fit(X, y)

In [ ]:
LinearSVC.

In [ ]:
%matplotlib inline

### 2.4 Tables describing Parameter search of `C` and `sigma`

In [ ]:
%%capture
CRange = {.01, .1, 1, 10, 100}
SRange = {.001, .01, .1, 1, 10}

c_gauss_results = pd.concat([
        pd.DataFrame({d:get_stats(d, kernel=funcy.partial(gaussian_kernel, sigma=s), C=c) 
                      for d in [1,2,3,4]}).T.assign(c=c, signma=s)
        for c in CRange
        for s in SRange
    ]).rename_axis(['data'])



In [ ]:
%%capture
c_lin_results = pd.concat([
        pd.DataFrame({d:get_stats(d,C=c) for d in [1,2,3,4]}).T.assign(c=c)
        for c in CRange]).rename_axis(['data'])

In [ ]:
c_lin_results.loc[1].iloc[-1].clf.C

In [ ]:
c_lin_results.set_index('c', append=True).sort_index()

In [ ]:
c_gauss_results.set_index('c', append=True).sort_index()

In [ ]:
show_plots(res_gauss)

# 3. Pegasos
**(2)** Observe the the margin (distance between the decision boundary and margin boundary) as a function of L

In [4]:
MAX_EPOCHS =1e4
def pegasos(X, y,L=2, max_epochs=MAX_EPOCHS):
    N = np.array([1. / (t*L) for t in range(1, int(MAX_EPOCHS *2))])
    w =np.zeros(X.shape[1])
    t = 0
    while t < max_epochs:
        for i, row in enumerate(X):
            t +=1
            w = (1 - N[t]*L) * w
            if y[i]* w.T.dot(row) < 1:
                w = w + N[t]*y[i]*row
    return w

class Pegasos(object):
    def __init__(self):
        pass
    def fit(self, X,y, **kwargs):
        self.coef_ = pegasos(X,y, **kwargs)
        return self
    def predict(self, X):
        return X.dot(self.coef_)


In [5]:
p = Pegasos().fit(X, y)
plotDecisionBoundary(X,y, p.predict, [-1,0,1], title='Pegasos SVM')

NameError: name 'X' is not defined

In [ ]:

plotDecisionBoundary(X[:4], y[:4], clf.predict, [-1,0,1])

In [ ]:
clf = LinearSVC().fit(X, y)
print clf.coef_
plotDecisionBoundary(X[:4], y[:4], clf.predict, [-1,0,1])

In [ ]:
pl.show()

In [ ]:
plotDecisionBoundary(X,np.array(y), clf.predict, [-1,0,1], title='Pegasos SVM, l={}'.format(L))

In [ ]:
pl.show()

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:

for power in range(-10, 1,2):
    L= 2^power
    p = Pegasos().fit(X, y, L=L)
    print p.coef_
    plotDecisionBoundary(X,y, p.predict, [-1,0,1], title='Pegasos SVM, l={}'.format(L))